In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Full_labeled_meme_dataset.csv')

# Check the contents of the dataframe
print("Dataframe head:\n", df.head())
print("Dataframe shape:", df.shape)

# Ensure the necessary columns exist
if 'image_name' not in df.columns or 'label' not in df.columns:
    raise ValueError("CSV file must contain 'image_name' and 'label' columns.")

# Define the path where images are stored
image_directory = '/content/drive/MyDrive/Colab Notebooks/Dataset/img1'

# Print the first few image paths to verify correctness
print("Sample image paths from CSV:")
image_paths = [os.path.join(image_directory, filename) for filename in df['image_name']]
print(image_paths[:5])

# Step 1: Load and preprocess the images
def load_image(filename):
    img_path = os.path.join(image_directory, filename)
    img = cv2.imread(img_path)  # Load image

    # Check if the image loaded correctly
    if img is None:
        print(f"Error loading image: {filename}")  # Log which images fail to load
        return None  # Return None to skip processing this image

    img = cv2.resize(img, (224, 224))  # Resize to 224x224 for consistency
    img = img / 255.0  # Normalize pixel values between 0 and 1
    return img

# Load all images into a list, filtering out None values
X = []
failed_images = []
for filename in df['image_name']:
    img = load_image(filename)
    if img is not None:
        X.append(img)
    else:
        failed_images.append(filename)

# Report failed image loads
if failed_images:
    print(f"Warning: {len(failed_images)} images failed to load.")
    print("Failed image filenames:", failed_images)

# Convert the list to a NumPy array
X = np.array(X)

# Labels (targets) - Check shape before one-hot encoding
y = df['label'].values[:len(X)]
print(f"Initial labels shape: {y.shape}")

# One-hot encoding if labels are categorical
y = to_categorical(y, num_classes=2)
print(f"One-hot encoded labels shape: {y.shape}")

# Debug info: Check if the dataset is non-empty
print(f"Number of successfully loaded images: {len(X)}")
print(f"Number of labels: {len(y)}")

# Step 2: Split into train, validation, and test sets if X and y are not empty
if len(X) > 0 and len(y) > 0:
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
else:
    raise ValueError("Dataset is empty. Please check data loading and preprocessing steps.")

# List a few files in the image directory for verification
print("Files in image directory:", os.listdir(image_directory)[:5])  # Lists a few files in the directory

# Step 3: Data Augmentation to prevent overfitting
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the data generator on the training data
train_datagen.fit(X_train)

# Now you can use the augmented training data in your model training pipeline.




Dataframe head:
   image_name                                     extracted_text  label
0  98034.png                                     No text found.      1
1  97805.png  excuse the shit out of\nHTT Ue Ce a\n(id you j...      1
2  98067.png  and i bring the white Pee\nPSE Ee TTS\n\nWCE E...      1
3  97845.png                                     No text found.      1
4  97815.png  manditory vaccinations mT “«:\n‘ exemptions Lo...      1
Dataframe shape: (2567, 3)
Sample image paths from CSV:
['/content/drive/MyDrive/Colab Notebooks/Dataset/img1/98034.png', '/content/drive/MyDrive/Colab Notebooks/Dataset/img1/97805.png', '/content/drive/MyDrive/Colab Notebooks/Dataset/img1/98067.png', '/content/drive/MyDrive/Colab Notebooks/Dataset/img1/97845.png', '/content/drive/MyDrive/Colab Notebooks/Dataset/img1/97815.png']
Initial labels shape: (2567,)
One-hot encoded labels shape: (2567, 2)
Number of successfully loaded images: 2567
Number of labels: 2567
Files in image directory: ['98034.png',

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

# Function to train and plot accuracy
def train_and_plot(model, X_train, y_train, X_val, y_val, epochs=60):

    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), verbose=2)
    # Calculate and plot confusion matrix
    y_pred = np.argmax(model.predict(X_val), axis=1)
    y_true = np.argmax(y_val, axis=1)
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, classes=['HATEFUL','NON HATEFUL'])
    plt.show()

    # Plot training history
    plot_training_history(history)


def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig('history_plot.png')
    plt.show()


    plt.tight_layout()


def plot_confusion_matrix(cm, classes, figsize=(8, 6)):
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Add, Flatten, Dense
from tensorflow.keras.regularizers import l2
def residual_block(x, filters, kernel_size, strides=1):
    shortcut = x

    x = Conv2D(filters, kernel_size, strides=strides, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    if strides != 1 or x.shape[-1] != shortcut.shape[-1]:
        shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same', kernel_initializer='he_normal')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = ReLU()(x)

    return x

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, Add

def residual_block(input_tensor, filters, kernel_size, strides=(1, 1)):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same', kernel_initializer='he_normal')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    # Skip connection
    shortcut = input_tensor
    if strides != (1, 1) or input_tensor.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same', kernel_initializer='he_normal')(input_tensor)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = ReLU()(x)

    return x

def create_resnet50_model(input_shape):
    input_tensor = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', kernel_initializer='he_normal')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Residual blocks with varying filter sizes
    x = residual_block(x, 64, (3, 3))
    x = residual_block(x, 64, (3, 3))
    x = residual_block(x, 64, (3, 3))

    x = residual_block(x, 128, (3, 3), strides=(2, 2))
    x = residual_block(x, 128, (3, 3))
    x = residual_block(x, 128, (3, 3))

    x = residual_block(x, 256, (3, 3), strides=(2, 2))
    x = residual_block(x, 256, (3, 3))
    x = residual_block(x, 256, (3, 3))

    x = residual_block(x, 512, (3, 3), strides=(2, 2))
    x = residual_block(x, 512, (3, 3))
    x = residual_block(x, 512, (3, 3))

    # Global average pooling
    x = GlobalAveragePooling2D()(x)

    # Dense layers
    x = Dense(1024, kernel_initializer='he_normal')(x)
    x = ReLU()(x)
    x = Dropout(0.5)(x)

    output_tensor = Dense(2, activation='softmax')(x)

    model = models.Model(inputs=input_tensor, outputs=output_tensor)

    return model

# Compile the model with a reduced learning rate for better convergence
model = create_resnet50_model((224, 224, 3))
optimizer = Adam(learning_rate=1e-4)  # Smaller learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Input shape
input_shape=(224,224, 3)

In [ ]:

# Create resnet model
resnet_model = create_resnet50_model(input_shape)
resnet_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_28 (Conv2D)        │ (None, 112, 112, 64)   │          9,472 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_28    │ (None, 112, 112, 64)   │            256 │ conv2d_28[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_26 (ReLU)           │ (None, 112, 112, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 56, 56, 64)     │              0 │ re_lu_26[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_29 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_29    │ (None, 56, 56, 64)     │            256 │ conv2d_29[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_27 (ReLU)           │ (None, 56, 56, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_30 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ re_lu_27[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_30    │ (None, 56, 56, 64)     │            256 │ conv2d_30[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_12 (Add)              │ (None, 56, 56, 64)     │              0 │ batch_normalization_3… │
│                           │                        │                │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_28 (ReLU)           │ (None, 56, 56, 64)     │              0 │ add_12[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_31 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ re_lu_28[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_31    │ (None, 56, 56, 64)     │            256 │ conv2d_31[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_29 (ReLU)           │ (None, 56, 56, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_32 (Conv2D)        │ (None, 56, 56, 64)     │         36,928 │ re_lu_29[0][0]         │
├──────────────────────

 Total params: 17,994,754 (68.64 MB)

 Trainable params: 17,981,314 (68.59 MB)

 Non-trainable params: 13,440 (52.50 KB)

In [ ]:
from keras.utils import to_categorical

# Assuming y_train and y_val are your original target variables
# Check if y_train and y_val are already in categorical format (shape (samples, 2))
if len(y_train.shape) == 1: # Check if y_train has only one dimension
    y_train = to_categorical(y_train, num_classes=2)

# Remove the second dimension if y_val is already categorical
if len(y_val.shape) == 3: # Check if y_val has 3 dimensions
    y_val = y_val[:, 0, :] # Select the first element of the second dimension
# Otherwise, convert to categorical format
elif len(y_val.shape) == 1: # Check if y_val has only one dimension
    y_val = to_categorical(y_val, num_classes=2)

In [ ]:
# Compile the model
resnet_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_and_plot(resnet_model, X_train, y_train, X_val, y_val)

Epoch 1/60
57/57 - 843s - 15s/step - accuracy: 0.8675 - loss: 0.7999 - val_accuracy: 0.9091 - val_loss: 41.7391
Epoch 2/60
